In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w6p1/sample_submission.csv
/kaggle/input/2022-ai-w6p1/pca_test.csv
/kaggle/input/2022-ai-w6p1/pca_train.csv
/kaggle/input/2022-ai-w6p1/train_label.csv


In [30]:
import random
import torch
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [31]:
trainX = pd.read_csv('../input/2022-ai-w6p1/pca_train.csv')
trainY = pd.read_csv('../input/2022-ai-w6p1/train_label.csv')
testX = pd.read_csv('../input/2022-ai-w6p1/pca_test.csv')
submit = pd.read_csv('../input/2022-ai-w6p1/sample_submission.csv')

In [32]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
trainX = scaler.fit_transform(trainX)
testX = scaler.transform(testX)

In [33]:
x_train = torch.FloatTensor(trainX).cuda()
y_train = torch.LongTensor(trainY['label']).cuda()
x_test = torch.FloatTensor(testX).cuda()

In [34]:
x_train.shape

torch.Size([27671, 256])

In [35]:
y_train.unique()

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 1

In [36]:
## NN 모델
n = x_train.shape[1]
m = 198
linear1 = torch.nn.Linear(n, m, bias=True).cuda()
linear2 = torch.nn.Linear(m, m, bias=True).cuda()
linear3 = torch.nn.Linear(m, m, bias=True).cuda()
sigmoid = torch.nn.Sigmoid()

In [37]:
model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid, linear3)

In [38]:
## 모덿학습 환경설정
loss = torch.nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [39]:
import torch.nn.functional as F

np_epochs = 10000
for epoch in range(np_epochs+1) :
    # gradient 초기화
    optimizer.zero_grad()
    # forward 계산
    hypothesis = model(x_train)
    # error 계산
    cost = F.cross_entropy(hypothesis, y_train)
    # backward 계산
    cost.backward()
    # 가중치 계산
    optimizer.step()
    
    if epoch % 1000 == 0:
        print(epoch, cost.item())

0 5.328965663909912
1000 4.734242916107178
2000 4.2191009521484375
3000 2.9527883529663086
4000 2.3944036960601807
5000 2.1356284618377686
6000 1.9682801961898804
7000 1.8509653806686401
8000 1.7621201276779175
9000 1.6910247802734375
10000 1.6316988468170166


In [40]:
with torch.no_grad():
    hypothesis = model(x_test)
    predict = torch.argmax(hypothesis, dim=1)

In [42]:
submit

,id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
13624,13624,0
13625,13625,0
13626,13626,0
13627,13627,0


In [44]:
submit['label'] = torch.Tensor.cpu(predict)

In [46]:
submit.to_csv('submit1.csv', index=False)